# Game theory solver notes

In [1]:
from game_theory import model

## Example with prisoner's dilemma

In [2]:
X = [[-1, -3], [0, -2]]
Y = [[-1, -3], [-3, -2]]

In [3]:
prison = model.game(name="Prisoner's Dilemma", payoffs=[X, Y])

In [4]:
prison

Prisoner's Dilemma

          A         B
A  [-1, -1]  [-3, -3]
B   [0, -3]  [-2, -2]

Nash Equilibrum(s) at: [[1, 1]]

## Example with stag hunt with irrelevant alternative

In [5]:
X = [[2, 0], [1, 1], [0, 0]]
Y = [[2, 1], [0, 1], [0,0 ]]

In [6]:
stag_hunt = model.game(name="Stag Hunt", payoffs=[X, Y])

In [7]:
stag_hunt

Stag Hunt

        A       B
A  [2, 2]  [0, 1]
B  [1, 0]  [1, 1]
C  [0, 0]  [0, 0]

Nash Equilibrum(s) at: [[0, 0], [1, 1]]

## Example of iterative, non-memory game

In [8]:
X = [[-1, -3], [0, -2]]
Y = [[-1, -3], [-3, -2]]
Z = [[2, 0], [1, 1]]
games = {"X X": [X, X], "X Y": [X, Y], "X Z": [X, Z],
         "Y Y": [Y, Y], "Y Z": [Y, Z], "Z Z": [Z, Z]}
number_of_games = 100
player_pop = {"X": 10, "Y": 5, "Z": 5}
env = model.evolution(games=games, number_of_games=number_of_games,
                      player_pop=player_pop)

In [9]:
env

[

X X

          A         B
A  [-1, -1]  [-3, -3]
B    [0, 0]  [-2, -2]

Nash Equilibrum(s) at: [[1, 0]], 

X Y

          A         B
A  [-1, -1]  [-3, -3]
B   [0, -3]  [-2, -2]

Nash Equilibrum(s) at: [[1, 1]], 

X Z

         A        B
A  [-1, 2]  [-3, 0]
B   [0, 1]  [-2, 1]

Nash Equilibrum(s) at: [[1, 0], [1, 1]], 

Y Y

          A         B
A  [-1, -1]  [-3, -3]
B  [-3, -3]  [-2, -2]

Nash Equilibrum(s) at: [[0, 0], [1, 1]], 

Y Z

         A        B
A  [-1, 2]  [-3, 0]
B  [-3, 1]  [-2, 1]

Nash Equilibrum(s) at: [[0, 0], [1, 1]], 

Z Z

        A       B
A  [2, 2]  [0, 0]
B  [1, 1]  [1, 1]

Nash Equilibrum(s) at: [[0, 0], [1, 1]]]

In [10]:
env.scores[0:5]

[{'X': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'Y': [0, 0, 0, 0, 0],
  'Z': [0, 0, 0, 0, 0]},
 {'X': [-2, -2, -2, 0, 0, 0, 0, 0, 0, 0],
  'Y': [-2, -2, -2, 2, 2],
  'Z': [2, -1, 1, -1, 2]},
 {'X': [-2, -4, -2, -2, -2, 1, 1, 0, 0, 0],
  'Y': [-3, -4, -4, 4, 0],
  'Z': [3, 1, 0, -1, 2]},
 {'X': [-2, -6, -2, -1, -4, 1, 1, 0, 0, -2],
  'Y': [-1, -6, -6, 3, -2],
  'Z': [5, 3, 0, 1, 1]},
 {'X': [-2, -8, -2, -3, -3, 1, 1, 0, 1, -2],
  'Y': [-3, -7, -4, 2, -4],
  'Z': [7, 3, -1, 1, 3]}]

## Rock, Papers, Scissors

In [11]:
X = [[0, -1, 1], [1, 0, -1], [-1, 1, 0]]
Y = [[0, 1, -1], [-1, 0, 1], [1, -1, 0]]
# rps = model.game(name="Rock, Papers, Scissors", payoffs=[X, Y])